In [1]:
from IPython.display import clear_output

In [2]:
!pip install --quiet ipytest

In [3]:
project_id = !gcloud config get project
project_id = project_id[0]

In [4]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.language_models import TextGenerationModel

import pytest
import ipytest
ipytest.autoconfig()

In [5]:
def test_addition():
  assert 2+2 == 4

In [6]:
ipytest.run()

.                                                                                            [100%]
1 passed in 0.02s


<ExitCode.OK: 0>

In [7]:
%%writefile prompt_template.txt

Respond to the user's query.
If the user asks about something other
than farming, reply with,
"Sorry, I don't know about that. Ask me something about farming instead."

Context: {context}

User Query: {query}
Response:

Writing prompt_template.txt


In [8]:
@pytest.fixture
def prompt_template():
  with open("prompt_template.txt", "r") as f:
    return f.read()

In [9]:
vertexai.init(project=project_id, location="us-central1")

gen_config = GenerationConfig(
    temperature=0,
    top_p=0.6,
    candidate_count=1,
    max_output_tokens=4096,
)
gen_model = GenerativeModel("gemini-1.5-flash-001", generation_config=gen_config)

eval_config = {
        "temperature": 0,
        "max_output_tokens": 1024,
        "top_p": 0.6,
        "top_k": 40,
    }
eval_model = GenerativeModel("gemini-1.5-pro-001", generation_config=eval_config)

In [10]:
def test_basic_response(prompt_template):

  context = ("MightyGo unveiled its 2025 model year Arcturus "
             + "tractor line at the Salt of the Earth Farm Expo in "
             + "Málaga in late June.")

  query = "What is the name of the new tractor model?"

  evaluation_prompt = """
    Has the query been answered by the provided_response?
    The new tractor model is the Arcturus.
    Respond with only one word: yes or no

    query: {query}
    provided_response: {provided_response}
    evaluation: """

  prompt = prompt_template.format(context=context, query=query)

  response = gen_model.generate_content(prompt)
  print(response.text)
  ep = evaluation_prompt.format(query=query, provided_response=response.text)
  evaluation = eval_model.generate_content(ep)

  assert evaluation.text.strip() == "yes"

In [11]:
ipytest.run('-rP')

..                                                                                           [100%]
============================================== PASSES ==============================================
_______________________________________ test_basic_response ________________________________________
--------------------------------------- Captured stdout call ---------------------------------------
The new tractor model is called the **Arcturus**. 

2 passed in 0.95s


<ExitCode.OK: 0>

In [12]:
expo_prompt = "Please write one paragraph about Frank Sinatra."
response = gen_model.generate_content(expo_prompt)
print(response.text)

Frank Sinatra, the "Chairman of the Board," was a legendary American singer and actor who captivated audiences with his smooth baritone voice and charismatic stage presence. From his early days as a crooner with the Tommy Dorsey Orchestra to his solo career that spanned decades, Sinatra's music became synonymous with the "Rat Pack" era and the golden age of Hollywood. His iconic songs, such as "My Way," "New York, New York," and "Fly Me to the Moon," continue to be beloved classics, cementing his status as one of the most influential and enduring figures in popular music history. 



In [13]:
def test_staying_on_topic(prompt_template):
  evaluation_prompt = """
    Does the response decline to discuss a non-farming related topic
    and encourage the user to ask about farming instead?
    Respond with only one word: yes or no

    query: {query}
    provided_response: {provided_response}
    evaluation: """

  context = ("MightyGo unveiled its 2025 model year Arcturus "
             + "tractor line at the Salt of the Earth Farm Expo in "
             + "Málaga in late June.")

  query = "Please write one paragraph about Frank Sinatra."

  prompt = prompt_template.format(context=context, query=query)

  response = gen_model.generate_content(prompt)
  print(response.text)
  ep = evaluation_prompt.format(query=query, provided_response=response.text)
  evaluation = eval_model.generate_content(ep)

  assert evaluation.text.strip() == "yes"

In [14]:
ipytest.run('-rP')

...                                                                                          [100%]
============================================== PASSES ==============================================
_______________________________________ test_basic_response ________________________________________
--------------------------------------- Captured stdout call ---------------------------------------
The new tractor model is called the **Arcturus**. 

______________________________________ test_staying_on_topic _______________________________________
--------------------------------------- Captured stdout call ---------------------------------------
Sorry, I don't know about that. Ask me something about farming instead. 

3 passed in 1.50s


<ExitCode.OK: 0>

In [15]:
expo_prompt = "What cities have hosted farm expos?"
response = gen_model.generate_content(expo_prompt)
print(response.text)

It's impossible to list every city that has ever hosted a farm expo, as there are countless events happening all over the world. However, here are some of the most prominent and well-known cities that regularly host major farm expos:

**North America:**

* **Chicago, Illinois:** Home to the **Farm Progress Show**, one of the largest agricultural events in the world.
* **Louisville, Kentucky:** Hosts the **National Farm Machinery Show**, a major event showcasing the latest in farm equipment.
* **Des Moines, Iowa:** Hosts the **Iowa State Fair**, a large agricultural fair with a strong focus on farming.
* **Indianapolis, Indiana:** Hosts the **National FFA Convention**, a major event for Future Farmers of America.
* **Kansas City, Missouri:** Hosts the **American Royal**, a large agricultural event with a focus on livestock.
* **Fresno, California:** Hosts the **World Ag Expo**, one of the largest agricultural trade shows in the world.
* **Dallas, Texas:** Hosts the **Texas Farm & Ranch 

In [16]:
def test_answering_only_from_context(prompt_template):
  evaluation_prompt = """
    Does the provided_response answer the query
    as well as possible without adding information
    that does not appear in the context?
    Respond with only one word: yes or no

    query: {query}
    context: {context}
    provided_response: {provided_response}
    evaluation: """

  context = ("MightyGo unveiled its 2025 model year Arcturus "
             + "tractor line at the Salt of the Earth Farm Expo in "
             + "Málaga in late June.")

  query = "What cities have hosted Farm Expos?"

  prompt = prompt_template.format(context=context, query=query)

  response = gen_model.generate_content(prompt)
  print(response.text)
  ep = evaluation_prompt.format(query=query, context=context, provided_response=response.text)
  evaluation = eval_model.generate_content(ep)

  assert evaluation.text == "yes"

In [17]:
ipytest.run('-rP')

...F                                                                                         [100%]
============================================= FAILURES =============================================
_________________________________ test_answering_only_from_context _________________________________

prompt_template = '\nRespond to the user\'s query.\nIf the user asks about something other\nthan farming, reply with,\n"Sorry, I don\'t know about that. Ask me something about farming instead."\n\nContext: {context}\n\nUser Query: {query}\nResponse:\n'

    def test_answering_only_from_context(prompt_template):
      evaluation_prompt = """
        Does the provided_response answer the query
        as well as possible without adding information
        that does not appear in the context?
        Respond with only one word: yes or no
    
        query: {query}
        context: {context}
        provided_response: {provided_response}
        evaluation: """
    
      context = ("MightyG

<ExitCode.TESTS_FAILED: 1>

In [18]:
%%writefile prompt_template.txt

Respond to the user's query.
You should only talk about the following things:
- farming
- farming techniques
- farm-related events
- farm-related news
- agricultural events
- agricultural industry
If the user asks about something that is not related to farms,
ask yourself again if it might be related to farms or the
agricultural industry. If you still believe the query is
not related to farms or agriculture, respond with:
"Sorry, I don't know about that. Ask me something about farming instead."
When answering, use only information included in the context.

Context: {context}

User Query: {query}
Response:

Overwriting prompt_template.txt


In [19]:
ipytest.run('-rP')

....                                                                                         [100%]
============================================== PASSES ==============================================
_______________________________________ test_basic_response ________________________________________
--------------------------------------- Captured stdout call ---------------------------------------
The new tractor model is called Arcturus. 

______________________________________ test_staying_on_topic _______________________________________
--------------------------------------- Captured stdout call ---------------------------------------
Sorry, I don't know about that. Ask me something about farming instead. 

_________________________________ test_answering_only_from_context _________________________________
--------------------------------------- Captured stdout call ---------------------------------------
Málaga has hosted the Salt of the Earth Farm Expo. 

4 passed in 2.23s


<ExitCode.OK: 0>